# Decision Tree Classifier 
---

In this notebook, we implement and evaluate a **Decision Tree Classifier** using the Spotify Top Songs dataset.  
This follows the same workflow as all previous models in your repo:

1. **Data Loading & Inspection**  
2. **Exploratory Analysis**  
3. **Train/Test Split**  
4. **Decision Tree Training**  
5. **Model Evaluation**  
6. **Decision Boundary Visualization (2D projection)**  
7. **Confusion Matrix & Performance Metrics**

Unlike previous notebooks, Decision Trees:

- Do **not require feature scaling**  
- Naturally handle **nonlinear splits**  
- Are interpretable via **tree diagrams**

This notebook uses the same 9 numerical Spotify features: danceability, energy, loudness, speechiness,
acousticness, instrumentalness, liveness,
valence, tempo

Our binary target is: popularity_above_median (1 = popular, 0 = not popular)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("topsongs.csv")

print(df.shape)
df.head()

(2000, 18)


,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


## Data Preparation

We will:

1. Select numerical audio features  
2. Extract the label (popularity ≥ 70 → hit)  
3. Standardize all features  
4. Train/test split

The final feature matrix is 9-dimensional.

In [6]:
# Define features manually based on your dataset
features = [
    "danceability", "energy", "loudness", "speechiness", "acousticness",
    "instrumentalness", "liveness", "valence", "tempo"
]

X = df[features].values
y = (df["popularity"] >= 70).astype(int).values  # binary hit label

# Standardize
X = (X - X.mean(axis=0)) / X.std(axis=0)

print("Feature matrix shape:", X.shape)
print("Positive class proportion:", y.mean())

Feature matrix shape: (2000, 9)
Positive class proportion: 0.3635


In [8]:
np.random.seed(42)
idx = np.random.permutation(len(X))

split = int(0.8 * len(X))

train_idx = idx[:split]
test_idx = idx[split:]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

print("Train:", X_train.shape, " Test:", X_test.shape)

Train: (1600, 9)  Test: (400, 9)


## Entropy and Information Gain

We define entropy as:

\[
H(S) = - \sum_{c \in classes} p(c) \log_2 p(c)
\]

Information Gain for splitting dataset \( S \) on feature \( f \) at threshold \( t \):

\[
IG = H(S) - \left( \frac{|S_L|}{|S|}H(S_L) + \frac{|S_R|}{|S|}H(S_R) \right)
\]

The tree will search:
- all features  
- multiple candidate thresholds  
to find the best split maximizing information gain.

In [11]:
def entropy(y):
    if len(y) == 0:
        return 0
    p = np.bincount(y) / len(y)
    p = p[p > 0]
    return -np.sum(p * np.log2(p))

def information_gain(y, left, right):
    H = entropy(y)
    HL = entropy(y[left])
    HR = entropy(y[right])
    wL = len(left) / len(y)
    wR = len(right) / len(y)
    return H - (wL * HL + wR * HR)

## Decision Tree Structure

The tree is implemented as a recursive class:

- If node is pure → create leaf  
- Else:
  - Try every feature
  - Try multiple thresholds
  - Choose the best split by entropy information gain  
  - Recurse left and right

In [14]:
class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # for leaf

In [16]:
class DecisionTreeScratch:

    def __init__(self, max_depth=10, min_samples=5):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.root = None

    ### Fit Model ###
    def fit(self, X, y):
        self.root = self._grow_tree(X, y, depth=0)

    ### Recursive tree builder ###
    def _grow_tree(self, X, y, depth):

        # Leaf conditions
        if len(np.unique(y)) == 1:
            return TreeNode(value=int(y[0]))
        if depth >= self.max_depth or len(y) < self.min_samples:
            return TreeNode(value=int(np.bincount(y).argmax()))

        n_samples, n_features = X.shape
        best_gain = -1
        best_feature = None
        best_threshold = None
        best_left = None
        best_right = None

        # Search best split
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for t in thresholds:
                left = np.where(X[:, feature] <= t)[0]
                right = np.where(X[:, feature] > t)[0]

                if len(left) == 0 or len(right) == 0:
                    continue

                gain = information_gain(y, left, right)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = t
                    best_left = left
                    best_right = right

        if best_gain == -1:
            return TreeNode(value=int(np.bincount(y).argmax()))

        left_child = self._grow_tree(X[best_left], y[best_left], depth+1)
        right_child = self._grow_tree(X[best_right], y[best_right], depth+1)

        return TreeNode(
            feature=best_feature,
            threshold=best_threshold,
            left=left_child,
            right=right_child
        )

    ### Prediction ###
    def predict_single(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self.predict_single(x, node.left)
        return self.predict_single(x, node.right)

    def predict(self, X):
        return np.array([self.predict_single(x, self.root) for x in X])

## Train the Decision Tree

In [19]:
tree = DecisionTreeScratch(max_depth=8, min_samples=10)
tree.fit(X_train, y_train)

print("Training complete.")

Training complete.


## Predictions and Evaluation

We compute:

- Accuracy  
- Confusion Matrix

In [22]:
y_pred = tree.predict(X_test)

accuracy = (y_pred == y_test).mean()
print("Accuracy:", round(accuracy, 4))

# Confusion matrix
cm = np.zeros((2,2), dtype=int)
for yt, yp in zip(y_test, y_pred):
    cm[yt, yp] += 1

print("\nConfusion Matrix:")
print(cm)

Accuracy: 0.5925

Confusion Matrix:
[[205  49]
 [114  32]]


## Text-Based Tree Visualization


In [25]:
def print_tree(node, depth=0):
    indent = "  " * depth
    if node.value is not None:
        print(f"{indent}Leaf → class {node.value}")
        return
    print(f"{indent}[X{node.feature} <= {node.threshold:.3f}]")
    print_tree(node.left, depth+1)
    print_tree(node.right, depth+1)

print_tree(tree.root)

[X7 <= 0.151]
  [X3 <= -0.523]
    [X4 <= -0.742]
      Leaf → class 0
      [X5 <= -0.162]
        [X0 <= -2.895]
          Leaf → class 0
          [X6 <= -0.747]
            [X0 <= -1.670]
              Leaf → class 1
              [X8 <= -0.561]
                Leaf → class 0
                Leaf → class 0
            [X6 <= -0.094]
              [X3 <= -0.802]
                Leaf → class 0
                Leaf → class 0
              [X3 <= -0.760]
                Leaf → class 0
                Leaf → class 0
        [X4 <= -0.733]
          Leaf → class 0
          [X4 <= -0.730]
            Leaf → class 1
            [X4 <= -0.705]
              Leaf → class 0
              [X7 <= -1.534]
                Leaf → class 0
                Leaf → class 0
    [X1 <= 1.648]
      [X2 <= 1.661]
        [X6 <= -0.710]
          [X7 <= 0.142]
            [X0 <= -1.820]
              Leaf → class 0
              [X8 <= 1.390]
                Leaf → class 0
                Leaf → class 1
 